In [28]:
import numpy as np

#measuring units for length is m
#define the size of the domain
L = 10;
H = 2

del_l = 0.1; #size of CV side

#number of cells in each direction
N_x = int(L/del_l);
N_y = int(H/del_l);

print(N_x, N_y)

100 20


In [29]:
#define the arrays for the scalar CV
#define arrays for storing velocities in a forward staggered arrangement

scalar_CV = np.zeros((1, N_x*N_y));
vel_u = np.zeros((1, (N_x + 1)*N_y));
vel_v = np.zeros((1, N_x*(N_y + 1))); #needed for completeness

In [30]:
#define the velocity field

#v_velocity field is 0

#u velocity field is a linear field from u_0 - first cell- to 0 - last cell -

u_0 = 0;
profile_u_1D = np.linspace(u_0, 0, (N_x +1));

for i in range(0, N_x):
    a = i*N_y;
    b = a + N_y;
    vel_u[0, a:b] = profile_u_1D[i];

print(profile_u_1D)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]


In [31]:
#computation of the fows

rho = 1; #g/cm^3
heat_c = 4.182; #J/gK at 25 deg C
A = del_l**2;

#u-flow
F_u = vel_u*rho*heat_c;
#v-flow
F_v = vel_v*rho*heat_c; #for completeness

#thermal condutivity of water
kappa = 0.589; #W/mK at 20 deg C
#NB the conductivity is subject to variation with temperature

#diffusive thermal conductance
D = (kappa/del_l);

print(D)
print(F_u)
print(F_v)

5.89
[[0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]]


In [32]:
#implementation of BC

#inlet and outlet BC for scalar CV 
#the velocity field is alrady given
#ignore pressure variations
mat_1 = np.zeros((1, N_y));
mat_2 = np.zeros((1, N_y));

#inlet temperature
mat_1[0, :] = 303;
inlet_BC = mat_1;
#outlet temperature
mat_2[0, :] = 273;
outlet_BC = mat_2;

#wall BC, constant heat flux out of the domain
heat_F_b = 0/A;
heat_F_t = 0/A; #careful with the signs
#these variables will be needed to deal with the source terms

print(inlet_BC)
print(outlet_BC)

[[303. 303. 303. 303. 303. 303. 303. 303. 303. 303. 303. 303. 303. 303.
  303. 303. 303. 303. 303. 303.]]
[[273. 273. 273. 273. 273. 273. 273. 273. 273. 273. 273. 273. 273. 273.
  273. 273. 273. 273. 273. 273.]]


In [33]:
#DOMAIN CREATION
#matrix construction, we are going to use the formalism of a sparsely occupied matrix
#we use CD for boundary cells
#we use HYBRID scheme for interior cells

#sparsely occupied matrix
#each row of the the hiantic matrix contains at most five elements 
#so we can in genreal arrange the sparsely occupied fomalism in the following way:
#first column goes the row, from 1 to N_y*N_x (all cells)
#second column goes the column in the matrix, from 1 to 5
#third column goes the coeffcient

#we use if-clauses to contruct the geometry
#next time you can also try the "in" command

sol_mat = np.zeros((N_x*N_y*5, 3));

inlet_cells = np.linspace(0, N_y - 1, N_y);
outlet_cells = np.linspace((N_x -1)*N_y, N_x*N_y -1, N_y);
bottom_wall_cells = (np.linspace(0,(N_x -1)*N_y, N_x ));
top_wall_cells = (np.linspace(N_y - 1, N_x*N_y - 1, N_x ));

for N in range(0, N_x*N_y):
    a = N*5;
    b = a + 5;
    sol_mat[a:b, 0] = N; #already the counting system in py

#define all cells
for M in range(0, N_x*N_y):
    a = M*5;
    
    sol_mat[a, 1] = M - 1; #south
    sol_mat[a + 1, 1] = M - N_y; #west
    sol_mat[a + 2, 1] = M; #P
    sol_mat[a + 3, 1] = M + N_y; #east
    sol_mat[a + 4, 1] = M + 1; #north

#adjust walls
for M in range(0, N_x*N_y):
    a = M*5;  
    
    for t in range(0, N_x):
        if M == bottom_wall_cells[t]:
            sol_mat[a, 1] = 0; #south
            sol_mat[a + 1, 1] = M - N_y; #west
            sol_mat[a + 2, 1] = M; #P
            sol_mat[a + 3, 1] = M + N_y; #east
            sol_mat[a + 4, 1] = M + 1; #north
        elif M == top_wall_cells[t]:
            sol_mat[a, 1] = M - 1; #south
            sol_mat[a + 1, 1] = M - N_y; #west
            sol_mat[a + 2, 1] = M; #P
            sol_mat[a + 3, 1] = M + N_y; #east
            sol_mat[a + 4, 1] = 0; #north

#adjust inlet and outlet
for M in range(0, N_x*N_y):
    a = M*5;
    
    for p in range(0, N_y):
        if M == inlet_cells[p]:
            sol_mat[a, 1] = M - 1; #south
            sol_mat[a + 1, 1] = 0; #west
            sol_mat[a + 2, 1] = M; #P
            sol_mat[a + 3, 1] = M + N_y; #east
            sol_mat[a + 4, 1] = M + 1; #north
        elif M == outlet_cells[p]:
            sol_mat[a, 1] = M - 1; #south
            sol_mat[a + 1, 1] = M - N_y; #west
            sol_mat[a + 2, 1] = M; #P
            sol_mat[a + 3, 1] = 0; #east
            sol_mat[a + 4, 1] = M + 1; #north

#adjust corners
for M in range(0, N_x*N_y):
    a = M*5;
    
    if M == 0:
        sol_mat[a, 1] = 0; #south
        sol_mat[a + 1, 1] = 0; #west
        sol_mat[a + 2, 1] = M; #P
        sol_mat[a + 3, 1] = M + N_y; #east
        sol_mat[a + 4, 1] = M + 1; #north
    elif M == (N_y - 1):
        sol_mat[a, 1] = M - 1; #south
        sol_mat[a + 1, 1] = 0; #west
        sol_mat[a + 2, 1] = M; #P
        sol_mat[a + 3, 1] = M + N_y; #east
        sol_mat[a + 4, 1] = 0; #north
    elif M == (N_x - 1)*N_y:
        sol_mat[a, 1] = 0; #south
        sol_mat[a + 1, 1] = M - N_y; #west
        sol_mat[a + 2, 1] = M; #P
        sol_mat[a + 3, 1] = 0; #east
        sol_mat[a + 4, 1] = M + 1; #north
    elif M == N_x*N_y -1:
        sol_mat[a, 1] = M - 1; #south
        sol_mat[a + 1, 1] = M - N_y; #west
        sol_mat[a + 2, 1] = M; #P
        sol_mat[a + 3, 1] = 0; #east
        sol_mat[a + 4, 1] = 0; #north
        

In [34]:
#calculation of the coeffcients and input in solution matrix (sol_mat)

#start from cells 0 and 1

#cell 0, apply CD
sol_mat[2, 2] = 3*D + (F_u[0, N_y] - F_u[0, 0] + F_v[0, 1])/2;
sol_mat[3, 2] = F_u[0, N_y]/2 - D;
sol_mat[4, 2] = F_v[0, 1]/2 - D;

#cell 1, apply CD
sol_mat[5, 2] = -F_v[0, 1]/2 - D;
sol_mat[7, 2] = 4*D + (F_u[0, 1 + N_y] - F_u[0, 1] + F_v[0, 2] - F_v[0, 1])/2;
sol_mat[8, 2] = F_u[0, 1 + N_y]/2 - D;
sol_mat[9, 2] = F_v[0, 2]/2 - D;

In [35]:
#HYBRID SCHEME IMPLEMENTATION
#loop for the calculation of all the other coefficients
#NB: even though we don't have these terms we DO HAVE their fluxes thanks to the staggered arrangement
#the computation process is expained on the paper notes

B_cells = np.r_[inlet_cells, bottom_wall_cells, top_wall_cells, outlet_cells]; #list of all B cells

for t in range(2, N_x*N_y):
    #define enumerators for the cells
    S = t*5;
    W = S + 1;
    P = W + 1;
    E = P + 1;
    N = E + 1;
    
    #must correct code for boundary cells, must consider fluxes 
    #############################################################################
    if (t in B_cells) == True:
        w = sol_mat[S : (S+5), 1];
        mask = np.ones((5, 1));
        mask[w == 0, 0 ] = 0;
        
        a_S = mask[0]*(-F_v[0, t]/2 - D);
        a_W = mask[1]*(-F_u[0, t]/2 - D);
        a_E = mask[3]*(F_u[0, t + N_y]/2 - D);
        a_N = mask[4]*(F_v[0, t + 1]/2 - D);
        a_P = (mask[3]*F_u[0, t + N_y] - mask[1]*F_u[0, t] + mask[4]*F_v[0, t + 1] - mask[0]*F_v[0, t] - a_E - a_N - a_W - a_S );
        
        sol_mat[S, 2] = mask[0]*a_S;
        sol_mat[W, 2] = mask[1]*a_W;
        sol_mat[P, 2] = a_P;
        sol_mat[E, 2] = mask[3]*a_E;
        sol_mat[N, 2] = mask[4]*a_N;
    #############################################################################

        
        #these lines of code below are completely general for all interior cells
    elif (t in B_cells) == False:
        Pe_u = abs((F_u[0, t]+F_u[0, t + N_y])/(2*D));
        Pe_v = abs((F_v[0, t]+F_v[0, t + 1])/(2*D));
        if Pe_u > 2 and Pe_v > 2: #full upwind
            if F_u[0, t] > 0 and F_u[0, t + N_y] > 0 and F_v[0, t] > 0 and F_v[0, t + 1] > 0:
                a_S = -F_v[0, t] - D;
                a_W = -F_u[0, t] - D;
                a_E = - D;
                a_N = - D;
                a_P = (F_u[0, t + N_y] - F_u[0, t] + F_v[0, t + 1] - F_v[0, t] - a_E - a_N - a_W - a_S );
                 
                sol_mat[S, 2] = a_S;
                sol_mat[W, 2] = a_W;
                sol_mat[P, 2] = a_P;
                sol_mat[E, 2] = a_E;
                sol_mat[N, 2] = a_N;
            elif F_u[0, t] > 0 and F_u[0, t + N_y] > 0 and F_v[0, t] < 0 and F_v[0, t + 1] < 0:
                a_S = - D;
                a_W = -F_u[0, t] - D;
                a_E = - D;
                a_N = F_v[0, t + 1] - D ;
                a_P = (F_u[0, t + N_y] - F_u[0, t] + F_v[0, t + 1] - F_v[0, t] - a_E - a_N - a_W - a_S );
                 
                sol_mat[S, 2] = a_S;
                sol_mat[W, 2] = a_W;
                sol_mat[P, 2] = a_P;
                sol_mat[E, 2] = a_E;
                sol_mat[N, 2] = a_N;
            elif F_u[0, t] < 0 and F_u[0, t + N_y] < 0 and F_v[0, t] > 0 and F_v[0, t + 1] > 0:
                a_S = -F_v[0, t] - D;
                a_W = - D;
                a_E = F_u[0, t + N_y] - D;
                a_N = - D ;
                a_P = (F_u[0, t + N_y] - F_u[0, t] + F_v[0, t + 1] - F_v[0, t] - a_E - a_N - a_W - a_S );
                 
                sol_mat[S, 2] = a_S;
                sol_mat[W, 2] = a_W;
                sol_mat[P, 2] = a_P;
                sol_mat[E, 2] = a_E;
                sol_mat[N, 2] = a_N;
            elif F_u[0, t] < 0 and F_u[0, t + N_y] < 0 and F_v[0, t] > 0 and F_v[0, t + 1] > 0:
                a_S = - D;
                a_W = - D;
                a_E = F_u[0, t + N_y] - D;
                a_N = F_v[0, t + 1] - D ;
                a_P = (F_u[0, t + N_y] - F_u[0, t] + F_v[0, t + 1] - F_v[0, t] - a_E - a_N - a_W - a_S );
                 
                sol_mat[S, 2] = a_S;
                sol_mat[W, 2] = a_W;
                sol_mat[P, 2] = a_P;
                sol_mat[E, 2] = a_E;
                sol_mat[N, 2] = a_N;
        elif Pe_u > 2 and Pe_v < 2: #upwind only in u-direction
            if F_u[0, t] > 0 and F_u[0, t + N_y] > 0:
                a_S = -F_v[0, t]/2 - D;
                a_W = -F_u[0, t] - D;
                a_E = - D;
                a_N = F_v[0, t + 1]/2 - D;
                a_P = (F_u[0, t + N_y] - F_u[0, t] + F_v[0, t + 1] - F_v[0, t] - a_E - a_N - a_W - a_S );
                 
                sol_mat[S, 2] = a_S;
                sol_mat[W, 2] = a_W;
                sol_mat[P, 2] = a_P;
                sol_mat[E, 2] = a_E;
                sol_mat[N, 2] = a_N;
            elif F_u[0, t] < 0 and F_u[0, t + N_y] < 0:
                a_S = -F_v[0, t]/2 - D;
                a_W = - D;
                a_E = F_u[0, t + N_y] - D;
                a_N = F_v[0, t + 1]/2 - D;
                a_P = (F_u[0, t + N_y] - F_u[0, t] + F_v[0, t + 1] - F_v[0, t] - a_E - a_N - a_W - a_S );
                 
                sol_mat[S, 2] = a_S;
                sol_mat[W, 2] = a_W;
                sol_mat[P, 2] = a_P;
                sol_mat[E, 2] = a_E;
                sol_mat[N, 2] = a_N;
        elif Pe_u < 2 and Pe_v > 2: #upwind only in v-direction
            if F_v[0, t] > 0 and F_v[0, t + 1] > 0:
                a_S = - F_v[0, t] - D;
                a_W = -F_u[0, t]/2 - D;
                a_E = F_u[0, t + N_y]/2 - D;
                a_N = - D;
                a_P = (F_u[0, t + N_y] - F_u[0, t] + F_v[0, t + 1] - F_v[0, t] - a_E - a_N - a_W - a_S );
                 
                sol_mat[S, 2] = a_S;
                sol_mat[W, 2] = a_W;
                sol_mat[P, 2] = a_P;
                sol_mat[E, 2] = a_E;
                sol_mat[N, 2] = a_N;
            elif F_v[0, t] < 0 and F_v[0, t + 1] < 0:
                a_S = - D;
                a_W = -F_u[0, t]/2 - D;
                a_E =  F_u[0, t + N_y]/2 - D;
                a_N = F_v[0, t + 1] - D;
                a_P = (F_u[0, t + N_y] - F_u[0, t] + F_v[0, t + 1] - F_v[0, t] - a_E - a_N - a_W - a_S );
                 
                sol_mat[S, 2] = a_S;
                sol_mat[W, 2] = a_W;
                sol_mat[P, 2] = a_P;
                sol_mat[E, 2] = a_E;
                sol_mat[N, 2] = a_N;
        elif Pe_u < 2 and Pe_v < 2: # full CD
            a_S = -F_v[0, t]/2 - D;
            a_W = -F_u[0, t]/2 - D;
            a_E = F_u[0, t + N_y]/2 - D;
            a_N = F_v[0, t + 1]/2 - D;
            a_P = (F_u[0, t + N_y] - F_u[0, t] + F_v[0, t + 1] - F_v[0, t] - a_E - a_N - a_W - a_S );
        
            sol_mat[S, 2] = a_S;
            sol_mat[W, 2] = a_W;
            sol_mat[P, 2] = a_P;
            sol_mat[E, 2] = a_E;
            sol_mat[N, 2] = a_N;

In [36]:
#correct for cell 50 that is to the right of cell 0
#apply CD
sol_mat[N_y*5 + 1, 2] = -F_u[0, N_y]/2 - D;

#for future reference, cells neighbouring the 0-cells require special treatment

In [38]:
import pandas as pd

dt = pd.DataFrame(sol_mat);
dt.to_excel("sol_matrix.xlsx")

In [25]:
#SOURCE TERM CREATION
#Convective source and diffusive source (forgot it in first istance)

b_source = np.zeros((1, N_x*N_y));

for M in range(0, N_x*N_y):
    a = 0;
    S = M*5;
    W = S + 1;
    E = W + 2;
    N = E + 1;
    
    if sol_mat[S, 1] == 0:
        a = - heat_F_b;
    if sol_mat[W, 1] == 0  and M != N_y: #adapted to the geometry of the problem
        t = int(sol_mat[W, 0]);
        a = a + (F_u[0, M]/2)*inlet_BC[0, t] + D*inlet_BC[0, t];
    if sol_mat[E, 1] == 0:
        t = int(sol_mat[E, 0]) -(N_x - 1)*N_y;
        a = a - (F_u[0, M + N_y]/2)*outlet_BC[0, t] + D*outlet_BC[0, t];
    if sol_mat[N, 1] == 0:
        a = a - heat_F_t;
    
    b_source[0, M] = a;


In [26]:
#dt = pd.DataFrame(b_source);
#dt.to_excel("source_vec.xlsx")

In [27]:
#Implementation of Gauss-Seidel method

#have a look to make sure that it is working properly
r = 101;
treshold = 100;

residual = 0;

while r > treshold:
    for i in range(0, N_x*N_y):
        S = i*5;
        W = S + 1;
        P = W + 1;
        E = P + 1;
        N = E + 1;
        
        s_1 = int(sol_mat[S, 1]);
        t_1 = (1/sol_mat[P, 2])*(sol_mat[S, 2]*scalar_CV[0, s_1]);
        w_2 = int(sol_mat[W, 1]);
        t_2 = (1/sol_mat[P, 2])*(sol_mat[W, 2]*scalar_CV[0, w_2]);
        e_4 = int(sol_mat[E, 1]);
        t_4 = (1/sol_mat[P, 2])*(sol_mat[E, 2]*scalar_CV[0, e_4]);
        n_5 = int(sol_mat[N, 1]);
        t_5 = (1/sol_mat[P, 2])*(sol_mat[N, 2]*scalar_CV[0, n_5]);
        
        S_u = (1/sol_mat[P, 2])*b_source[0, i];
        
        scalar_CV[0, i] = S_u - t_1 - t_2 - t_4 - t_5;
    
    for n in range(0, N_x*N_y):
        S = n*5;
        W = S + 1;
        P = W + 1;
        E = P + 1;
        N = E + 1;
        
        s_1 = int(sol_mat[S, 1]);
        t_1 = sol_mat[S, 2]*scalar_CV[0, s_1];
        w_2 = int(sol_mat[W, 1]);
        t_2 = sol_mat[W, 2]*scalar_CV[0, w_2];
        p_3 = int(sol_mat[P, 1]) #same, it is the diagonal
        t_3 = sol_mat[P, 2]*scalar_CV[0, p_3];
        e_4 = int(sol_mat[E, 1]);
        t_4 = sol_mat[E, 2]*scalar_CV[0, e_4];
        n_5 = int(sol_mat[N, 1]);
        t_5 = sol_mat[N, 2]*scalar_CV[0, n_5];
        
        S_u = b_source[0, i];
        
        residual = residual + abs(t_1 + t_2 + t_3 + t_4 - S_u);
    
    r = residual;
    
    print(scalar_CV[0, 0])

75.75
108.890625
131.181640625
149.23829481336807
165.064703953119
179.46755578717418
192.84847446125542
205.4399943459617
217.39176064466875
228.8075953968563
239.76359482005606
250.3178819814268
260.5162863864167
270.39586283463643
279.987183868237
289.3158962799745
298.4038150011396
307.26971454711435
315.9299160081107
324.39873182057937
332.68880917258576
340.8113996125415
348.7765738806038
356.5933953242472
364.27006144776857
371.81402054725504
379.2320685917283
386.53043025866697
393.7148271388728
400.7905354737992
407.7624353012435
414.6350525129793
421.41259503805964
428.0989841366503
434.6978816066797
441.21271355894305
447.6466912979864
454.0028297503655
460.2839638042705
466.492762861509
472.63174385162006
478.7032829162006
484.7096259375178
490.6528980576908
496.5351123119559
502.3581774808202
508.12390525048005
513.8340167581196
519.4901485880921
525.0938582761416
530.6466293713904
536.1498760995756
541.604947665714
547.0131322298765
552.3756605858941
557.6937095695089
562

1658.7679457068075
1660.5712663842478
1662.372191650386
1664.1707293110799
1665.9668871320262
1667.7606728390524
1669.5520941184025
1671.3411586170248
1673.1278739428537
1674.9122476650903
1676.6942873144799
1678.4740003835884
1680.2513943270756
1682.0264765619636
1683.7992544679073
1685.5697353874605
1687.3379266263385
1689.1038354536795
1690.8674691023043
1692.6288347689729
1694.3879396146374
1696.1447907646989
1697.8993953092522
1699.6517603033383
1701.4018927671877
1703.149799686465
1704.8954880125114


KeyboardInterrupt: 

In [ ]:
import pandas as pd
scalar_CV = scalar_CV.T
temperature_list = scalar_CV.tolist();

result = {
    "temperature": temperature_list
    
}

df = pd.DataFrame(result)

print(df)